In [ ]:
#| default_exp database

# Database 

I absolutely love [SQLAlchemy](link here). Data Modeling is one of the concepts that initially drew me to coding. But I hate creating the engine and and all the different options they give you for establishing a connection. And how easy it is to screw it up. Especially because 90% of the time I just want to preview a result in a DataFrame.

I wanted a simple wrapper class that handled instantiation and the context managers required for connecting to the engine. While we're at it we can make it look nice, too.

In [ ]:
#| export 

class Database():

    pass